# Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

# Download Library

In [ ]:
"""
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
# !pip install faiss-gpu
!pip install faiss-cpu
"""

# Import Library

In [1]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    TrainingArguments
)
from accelerate import Accelerator


from trl import SFTConfig, SFTTrainer
from datasets import load_dataset

#LORA
from peft import LoraConfig, PeftModel

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

c:\Users\qkdru\anaconda3\envs\python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vector DB

In [7]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


# DB 생성

In [8]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

# PDF 데이터베이스를 pickle 파일로 저장
with open('pdf_databases_cpu.pickle', 'wb') as f:
    pickle.dump(pdf_databases, f)

print("pdf_databases가 pickle 파일로 저장되었습니다.")

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]c:\Users\qkdru\anaconda3\envs\python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:04<00:36,  4.56s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:08<00:29,  4.19s/it]

...

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:37<00:05,  5.15s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:43<00:00,  4.85s/it]


pdf_databases가 pickle 파일로 저장되었습니다.


In [2]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
with open('pdf_databases_cpu.pickle', 'rb') as f:
    pdf_databases = pickle.load(f)

c:\Users\qkdru\anaconda3\envs\python310\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


# MODEL Import

In [ ]:
def print_model_layers(model):
    # 모델의 모든 레이어 이름과 구조 출력
    for name, module in model.named_modules():
        print(f"{name}: {module}")

# 모델 로드
model_id = "beomi/llama-2-ko-7b"
model = AutoModelForCausalLM.from_pretrained(model_id)

# 레이어 이름과 구조 출력
print_model_layers(model)

In [ ]:
# 특정 레이어 동결 및 학습 가능 설정
for name, param in model.named_parameters():
    print(name)

In [ ]:
def print_model_layers(model):
    # 모델의 모든 레이어 출력
    for name, module in model.named_modules():
        print(f"{name}: {module}")

# 모델 로드 후 레이어 출력
model = AutoModelForCausalLM.from_pretrained("beomi/gemma-ko-2b")
print_model_layers(model)

In [1]:
import pandas as pd

mydf = pd.read_csv("stf_train.csv")
print(mydf["Question"][2])
print(mydf["Answer"][2])

기금이 예산과 다른 점은?
기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정부가 편성하고 국회에서 심의・의결한 기금운용계획에 의해 운용됩니다.


In [2]:
def setup_llm_SFTTrainer():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID 
    #model_id = "beomi/gemma-ko-2b"
    model_id = "beomi/llama-2-ko-7b"
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    
    # Load LoRA configuration
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )        
        
    
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=3,
        optim="paged_adamw_8bit",
        learning_rate=1e-4,
        weight_decay=0.01,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        
        logging_dir='./logs',
        logging_steps=10,
        save_steps=1000,
        evaluation_strategy="steps",

    )


    #dataset
    train_dataset = load_dataset('csv', data_files='stf_train.csv')['train']
    eval_dataset = load_dataset('csv', data_files='stf_eval.csv')['train']  
    
    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example)):
            text = """
        당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.
        
        답변할 때에는 다음 예시와 같이 간결하게 답변해주십시오
        [예시1]
        2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.
        [예시2]
        2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.
        [예시3]
        기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정부가 편성하고 국회에서 심의・의결한 기금운용계획에 의해 운용됩니다.
        [예시4]
        사회보장기여금은 국가가 국민의 안정적인 삶을 보장하기 위해 운영하는 사회보장제도의 재정을 마련하기 위해 징수하는 금액입니다. 이 기여금은 국민의 노후, 실업, 질병, 장애 등 다양한 사회적 위험으로부터 보호하기 위한 목적을 가지고 있으며, 연금, 건강보험, 고용보험 등의 다양한 사회보장 프로그램에 필요한 자금으로 사용됩니다. 이를 통해 국민 개개인이 경제적, 사회적 어려움에 직면했을 때 필요한 지원을 받을 수 있도록 돕습니다.
        
        다음 정보를 바탕으로 질문에 답해주세요
        
        [정보]
        {example[Context]}

        [질문]
        {example[Question]}
        
        [답변]
        {example[Answer]}
        """
            
            
            output_texts.append(text)
        return output_texts

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        peft_config = peft_config,
        formatting_func=formatting_prompts_func,
        train_dataset = train_dataset,
        eval_dataset = eval_dataset,   
    )

    # Train model
    trainer.train()

    finetuned_model = "finetuned_model"
    # Save trained model
    trainer.model.save_pretrained(finetuned_model)
    
    text_generation_pipeline = pipeline(
        model=trainer.model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=128,
        
        do_sample=True,
        temperature=0.8,
        num_beams=3,
        top_p=1.0,
        top_k=5,
        epsilon_cutoff=9e-4,
        eta_cutoff=2e-3,
        penalty_alpha=1.0,
        repetition_penalty=1.2,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)
    return hf
    

In [3]:
# LLM 파이프라인
llm = setup_llm_SFTTrainer()

Loading checkpoint shards:  53%|█████▎    | 8/15 [00:04<00:03,  2.08it/s]

In [ ]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./test.csv')
with open('pdf_databases_cpu.pickle', 'rb') as f:
    pdf_databases = pickle.load(f)

c:\Users\qkdru\anaconda3\envs\python310\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


# Langchain 을 이용한 추론

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# 배치 사이즈 설정
batch_size = 2  # 원하는 배치 크기로 설정

# DataFrame의 각 행에 대해 처리
for start in tqdm(range(0, len(df), batch_size), desc="Answering Questions"):
    # 현재 배치 선택
    batch_rows = df.iloc[start:start + batch_size]

    # 배치 내의 각 행 처리
    for _, row in batch_rows.iterrows():
        # 소스 문자열 정규화
        source = normalize_string(row['Source'])
        question = row['Question']

        # 정규화된 키로 데이터베이스 검색
        normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
        retriever = normalized_keys[source]['retriever']

        # RAG 체인 구성
        template = """
        당신은 질문에 답변하는 역할을 하는 챗봇입니다. 사용자의 질문에 올바른 답변을 하세요.
        
        답변할 때에는 다음 예시와 같이 간결하게 답변해주십시오
        [예시1]
        2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.
        [예시2]
        2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.
        [예시3]
        기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정부가 편성하고 국회에서 심의・의결한 기금운용계획에 의해 운용됩니다.
        [예시4]
        사회보장기여금은 국가가 국민의 안정적인 삶을 보장하기 위해 운영하는 사회보장제도의 재정을 마련하기 위해 징수하는 금액입니다. 이 기여금은 국민의 노후, 실업, 질병, 장애 등 다양한 사회적 위험으로부터 보호하기 위한 목적을 가지고 있으며, 연금, 건강보험, 고용보험 등의 다양한 사회보장 프로그램에 필요한 자금으로 사용됩니다. 이를 통해 국민 개개인이 경제적, 사회적 어려움에 직면했을 때 필요한 지원을 받을 수 있도록 돕습니다.
        
        다음 정보를 바탕으로 질문에 답해주세요
        
        [정보]
        {context}

        [질문]
        {question}
        [답변]
        
        """
        prompt = PromptTemplate.from_template(template)

        # RAG 체인 정의
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        # 답변 추론
        print(f"Question: {question}")
        full_response = rag_chain.invoke(question)

        print(f"Answer: {full_response}\n")

        # 결과 저장
        results.append({
            "Source": row['Source'],
            "Source_path": row['Source_path'],
            "Question": question,
            "Answer": full_response
        })
        
        
        
        
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)

Answering Questions:   0%|          | 0/49 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2조 78억원입니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 1/49 [20:19<16:15:44, 1219.69s/it]

...

Answer: 재정정책에서 공적보증채무와 다른 일회성 보증은 다음과 같이 구분됩니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
Answer: 2024년 중앙정부 재정체계는 예산(일반・특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?


Answering Questions:  98%|█████████▊| 48/49 [16:37:49<21:04, 1264.73s/it]

Answer: 국가결산보고서는 국가회계기준에 따라 우발부채를 의무 이행을 위해 자원이 유출될 가능성이 희박​하지 않은 한 주석5 ‘우발사항 및 약정사항’에 공시하고 있으며, 재정상태표에는 인식하지 않음12)​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?
Answer: 우발부채는 미래에 특정 사건(들)이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 그 이상의 조건이 충​족되어야 금융거래로 인식된다는 점에서 ‘부채’와 차이6)​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 49/49 [16:59:23<00:00, 1248.24s/it]

Answer: 우발부채는 처음에 예상하지 못한 상황에 따라 변할 수 있으므로 지속적 검토가 필요하며, 과거 우발부채로 처리했더라도 경제적 편익 있는 자원의 유출가능성이 매우 높아진 경우, 그러한 가능성의 변화가 생긴 기간의 재정상태표에 ‘충당부채’로 인식​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​



# Submission

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)